In [5]:
# Import the necessary libraries

import re
from numpy import array
from numpy import asarray
from numpy import zeros

import tensorflow as tf
import keras
import keras.backend as K
from keras.models import Model
from keras.regularizers import L2
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Dropout, Input, LSTM
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping, ModelCheckpoint

import nltk
nltk.download('punkt')

from gensim import models

[nltk_data] Downloading package punkt to /home/raghav/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
# Helper function that cleans the input data and enumerates the labels

def extract(s):
    s = re.sub('\\(', '', s)
    s = re.sub('\\)', '', s)
    s = re.sub('\\s{2,}', ' ', s)
    return s.strip()

labels = {'entails\n': 0, 'contradiction': 1, 'neutral\n': 2}

In [7]:
# Function that reads data and parses data from file

def readFileData(filePath):
    with open(filePath, 'r') as f:
        inputRows = [row.split('\t') for row in f.readlines()[1:]]

    inputPremises = [row[0] for row in inputRows if row[2] in labels]
    inputHypotheses = [row[1] for row in inputRows if row[2] in labels]
    inputLabels = [labels[row[2]] for row in inputRows if row[2] in labels]
    f.close()

    return [inputPremises, inputHypotheses, inputLabels]

In [8]:
# Reading train and test data

trainData = readFileData('./scitail_1.0_train.tsv')
testData = readFileData('./scitail_1.0_test.tsv')
validationData = readFileData('./scitail_1.0_dev.tsv')

In [9]:
# Hyperparameters

maxLen = 32
epochs = 1000
batchSize = 512
gloveDimension = 300
hiddenDimension = 100
regularization = 4e-6

In [10]:
# Tokenizer to generate the vocabulary of the system

tokenizer = Tokenizer()
tokenizer.fit_on_texts(trainData[0] + trainData[1])
vocabSize = len(tokenizer.word_index)+1

In [11]:
# Convert the train data to sequences as per the vocabulary
trainData[0] = tokenizer.texts_to_sequences(trainData[0])
trainData[1] = tokenizer.texts_to_sequences(trainData[1])

# Pad or trim all generated sequences to the same max sentence length
trainData[0] = pad_sequences(trainData[0], maxLen, padding='post')
trainData[1] = pad_sequences(trainData[1], maxLen, padding='post')

# Transform the labels to one-hot encoding
trainData[2] = tf.keras.utils.to_categorical(trainData[2], num_classes=3)

In [12]:
# Convert the test data to sequences as per the vocabulary
testData[0] = tokenizer.texts_to_sequences(testData[0])
testData[1] = tokenizer.texts_to_sequences(testData[1])

# Pad or trim all generated sequences to the same max sentence length
testData[0] = pad_sequences(testData[0], maxLen, padding='post')
testData[1] = pad_sequences(testData[1], maxLen, padding='post')

# Transform the labels to one-hot encoding
testData[2] = tf.keras.utils.to_categorical(testData[2], num_classes=3)

In [13]:
# Convert the validation data to sequences as per the vocabulary
validationData[0] = tokenizer.texts_to_sequences(validationData[0])
validationData[1] = tokenizer.texts_to_sequences(validationData[1])

# Pad or trim all generated sequences to the same max sentence length
validationData[0] = pad_sequences(validationData[0], maxLen, padding='post')
validationData[1] = pad_sequences(validationData[1], maxLen, padding='post')

# Transform the labels to one-hot encoding
validationData[2] = tf.keras.utils.to_categorical(validationData[2], num_classes=3)

In [14]:
embeddingsDict = models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True)

In [15]:
# embeddingsDict = dict()
# glove = open(r'/content/drive/MyDrive/glove.840B.300d.txt', encoding='utf8')

# for line in glove:
#     records = line.split()
#     word = ''.join(records[:-300])
#     vectorDimensions = asarray(records[-300:], dtype='float32')
#     embeddingsDict[word] = vectorDimensions

# glove.close()

In [16]:
# Iterate through the embeddings and store only those that are present in our vocabulary
'''
embeddingsMat = zeros((vocabSize, gloveDimension))
for word, index in tokenizer.word_index.items():
    vec = embeddingsDict.get(word)
    if vec is not None:
        embeddingsMat[index] = vec
'''

embeddingsMat = zeros((vocabSize, gloveDimension))
for word, index in tokenizer.word_index.items():
    if index % 2500 == 0:
        print(index)
    if word in embeddingsDict.index_to_key:
        embeddingsMat[index] = embeddingsDict[word]

2500
5000
7500
10000
12500
15000
17500
20000


In [17]:
# Define the embedding layer for our baseline RNN model
embed = Embedding(vocabSize, gloveDimension, weights=[embeddingsMat], input_length=maxLen, trainable=False)

# As Premise and Hypothesis are distinct and are to be inputted separately, define two inputs and embed
premise = Input(shape=(maxLen,), dtype='int32')
hypothesis = Input(shape=(maxLen,), dtype='int32')

premInput = embed(premise)
hypoInput = embed(hypothesis)

convert = Dense(hiddenDimension, activation='tanh', input_shape=(gloveDimension,))

premInput = convert(premInput)
hypoInput = convert(hypoInput)

2022-05-01 16:41:52.909043: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-05-01 16:41:52.918072: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-01 16:41:52.918409: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-05-01 16:41:52.918612: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-05-01 16:41:52.918707: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

In [18]:
# Once the sentence embeddings have been generated, generate a matrix of dimensions maxLen X gloveDimension
# On adding maxLen, we get a single embedding vector of length gloveDimension

#rnn = keras.layers.core.Lambda(lambda x: K.sum(x, axis=1), output_shape=hiddenDimension)
#rnn = LSTM(hiddenDimension, dropout=0.2, recurrent_dropout=0.2)

rnnH = LSTM(hiddenDimension, dropout=0.2, return_state=True)
rnnP = LSTM(hiddenDimension, dropout=0.2)

In [19]:
# Apply batch normalization to the two input embeddings separately

premInput, sH, sC = rnnH(premInput)
hypoInput = rnnP(hypoInput, initial_state=[sH, sC])
premInput = tf.keras.layers.BatchNormalization()(premInput)
result = tf.keras.layers.BatchNormalization()(hypoInput)

In [20]:
# Joint is a concatenated embeddings layer, generated from the premise and hypothesis inputs
# Dilution of probability 0.2, to assist in regularization
#joint = keras.layers.concatenate([premInput, hypoInput])
result = Dropout(0.2)(result)
'''
for i in range(3):
    result = Dense(2*hiddenDimension, activation='tanh', kernel_regularizer=L2(regularization))(result)
    result = Dropout(0.2)(result)
    result = tf.keras.layers.BatchNormalization()(result)
'''

# 3 layers of the TanH activation function, along with L2 regularization.
# The final decision is based on the Softmax function

result = Dense(hiddenDimension, activation='tanh', kernel_regularizer=L2(regularization))(result)
pred = Dense(3, activation='softmax')(result)

In [21]:
# Defining the final models input and output format, as well as compilation parameters

model = Model(inputs=[premise, hypothesis], outputs=pred)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [26]:
# Fitting the model using the train data

callback = EarlyStopping(monitor='val_loss', min_delta=0, patience=20)
mcCallback = ModelCheckpoint('./2lstmsci', monitor='val_loss')
model.fit([array(trainData[0]), array(trainData[1])], array(trainData[2]), batch_size=batchSize, epochs=epochs, callbacks=[callback, mcCallback], validation_data=[[array(validationData[0]), array(validationData[1])], array(validationData[2])])

Epoch 1/1000
47/47 [==============================] - ETA: 0s - loss: 0.2502 - accuracy: 0.8958

INFO:tensorflow:Assets written to: ./2lstmsci/assets


INFO:tensorflow:Assets written to: ./2lstmsci/assets


47/47 [==============================] - 38s 820ms/step - loss: 0.2502 - accuracy: 0.8958 - val_loss: 0.7299 - val_accuracy: 0.6723
Epoch 2/1000
47/47 [==============================] - ETA: 0s - loss: 0.2454 - accuracy: 0.8988

INFO:tensorflow:Assets written to: ./2lstmsci/assets


...

47/47 [==============================] - 38s 806ms/step - loss: 0.2166 - accuracy: 0.9107 - val_loss: 0.8284 - val_accuracy: 0.6853
Epoch 8/1000
47/47 [==============================] - ETA: 0s - loss: 0.2149 - accuracy: 0.9116

INFO:tensorflow:Assets written to: ./2lstmsci/assets


INFO:tensorflow:Assets written to: ./2lstmsci/assets


47/47 [==============================] - 39s 823ms/step - loss: 0.2149 - accuracy: 0.9116 - val_loss: 0.8724 - val_accuracy: 0.6723
Epoch 9/1000
17/47 [=========>....................] - ETA: 20s - loss: 0.2027 - accuracy: 0.9131

KeyboardInterrupt: 

In [28]:
model.save('./2LSTMsci')

INFO:tensorflow:Assets written to: ./2LSTMsci/assets


INFO:tensorflow:Assets written to: ./2LSTMsci/assets


In [27]:
# Evaluating accuracy on the trained model

loss, acc = model.evaluate([array(testData[0]), array(testData[1])], array(testData[2]), batch_size=256)
print('Loss = ', loss)
print('Acc = ', acc)

9/9 [==============================] - 1s 135ms/step - loss: 0.9155 - accuracy: 0.7078
Loss =  0.9155178070068359
Acc =  0.7077646851539612


In [ ]:
ccc = keras.models.load_model('./32TwoLSTM')
l, a = ccc.evaluate([array(testData[0]), array(testData[1])], array(testData[2]), batch_size=256)
print(a)

39/39 [==============================] - 2s 28ms/step - loss: 0.5106 - accuracy: 0.8013
0.8013029098510742
